In [16]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
pd.options.display.max_columns = 90
pd.options.display.max_rows = 3000
pd.options.display.width = 5000
pd.options.mode.chained_assignment = None

stock_i = 'HK.800000'

dateList = np.load('Database/' + stock_i + '/date list.npy')

for i in dateList[:5]:
    dataset = pd.read_csv('Database/' + stock_i + '/Dailize/' + i + '.csv')
    dataset.drop(['change_rate'], axis=1, inplace=True)
    dataset['change'] = dataset['close'] - dataset['open']
    dataset['dif from ave'] = dataset['change'] - dataset['change'].rolling(30).mean()

    dataset['day high'] = ' '
    dataset['day low'] = ' '

    dataset['average'] = 0
    dataset['std'] = 0
    dataset['singal'] = None
    dataset['port'] = None
    average_positive, change_positive, positive_count, change_negative = 0, 0, 0, 0
    portfolio, holding, count, countS = 0, 0, 0, 0
    for j in range(30, len(dataset)-2):
        dataset['day high'][j] = dataset['high'][:j].max()
        dataset['day low'][j] = dataset['high'][:j].min()
        dataset['std'][j] = dataset['change'][:j].std()

        if dataset['change'][j] > 0:
            change_positive += dataset['change'][j]
            positive_count += 1
            dataset['average'][j] = change_positive / positive_count

            if dataset['change'][j] > dataset['std'][j] / 2 and dataset['open'][j] < dataset['day high'][j] - 50:
                dataset['singal'][j] = 'buy'
                if dataset['change'][j + 1] != None:
                    portfolio += dataset['change'][j + 1]
                    count += 1
        elif dataset['change'][j] < 0:
            change_negative += dataset['change'][j]

        dataset['port'][j] = portfolio


    #averPo = sum(dataset['change'] > 0) / dataset[dataset['change'] > 0].count()['change']
    #dataset['singal'] = dataset['dif from ave'].apply(lambda x: 'Y' if x > averPo else ' ')

    #dataset = dataset[['time_key', 'open', 'close', 'high', 'low', 'turnover']]

    print(dataset)
    print(portfolio, count*3)


                time_key      open     close      high       low      turnover  change  dif from ave  day high   day low  average  std singal   port
0    2020-01-02 09:30:00  28249.37  28249.37  28249.37  28249.37  7.728218e+08    0.00           NaN                            0    0   None   None
1    2020-01-02 09:31:00  28249.56  28265.32  28271.69  28245.97  1.003769e+09   15.76           NaN                            0    0   None   None
2    2020-01-02 09:32:00  28271.37  28303.00  28303.00  28263.98  6.171331e+08   31.63           NaN                            0    0   None   None
3    2020-01-02 09:33:00  28304.99  28338.10  28338.10  28304.99  4.995521e+08   33.11           NaN                            0    0   None   None
4    2020-01-02 09:34:00  28334.05  28340.32  28350.61  28331.52  6.052466e+08    6.27           NaN                            0    0   None   None
5    2020-01-02 09:35:00  28342.30  28334.37  28342.30  28325.85  6.081921e+08   -7.93           NaN      